In [1]:
import os
path_root = os.path.abspath('./')
if os.path.basename(path_root)=='py_scripts':
    path_root = os.path.abspath('../')
path_to_data = path_root+'/data/'    
path_sd_hawkes = os.path.dirname(path_root)+'/sd_hawkes_powerlaw/'
import numpy as np
import pandas as pd
import pickle
import datetime
import time

In [2]:
import sys
sys.path.append(path_root+'/py_scripts/')
sys.path.append(path_sd_hawkes+'/modelling/')
sys.path.append(path_sd_hawkes+'/resources/')

In [3]:
import model as sd_hawkes_model
import lob_model
import prepare_from_lobster as from_lobster

In [4]:
symbol='INTC'
initial_time=float(11.5*60*60)
final_time=float(15.0*60*60)
first_read_fromLOBSTER=True
dump_after_reading=False
add_level_to_messagefile=True

In [5]:
date='2019-01-31'

In [6]:
print('symbol={}, date={}'.format(symbol,date))
if (first_read_fromLOBSTER):
    LOB,messagefile=from_lobster.read_from_LOBSTER(symbol,date,
                                      dump_after_reading=dump_after_reading,
                                      add_level_to_messagefile=add_level_to_messagefile)
else:
    LOB,messagefile=from_lobster.load_from_pickleFiles(symbol,date)

LOB,messagefile=from_lobster.select_subset(LOB,messagefile,
                              initial_time=initial_time,
                              final_time=final_time)

symbol=INTC, date=2019-01-31
Given shape of orderbook: 1021687x40
I am dropping column 6 of messagefile. Please disregard any Warning about this column
given shape of messagefile: 1021687x6
read_from_LOBSTER: time window:
  initial_time=34200.007032624
  final_time=57599.991693465
I am adding 'level' to messagefile
declare_level: check = True


In [7]:
aggregate_time_stamp=True
eventTypes_to_aggregate=[1,2,3,4,5]
eventTypes_to_drop_with_nonunique_time=[]
eventTypes_to_drop_after_aggregation=[3]
only_4_events=False
separate_directions=True
separate_13_events=False
separate_31_events=False
separate_41_events=True
separate_34_events=True
separate_43_events=True
equiparate_45_events_with_same_time_stamp=True
drop_all_type3_events_with_nonunique_time=False
drop_5_events_with_same_time_stamp_as_4=True
drop_5_events_after_aggregation=True
tolerance_when_dropping=1.0e-8
add_hawkes_marks=True
clear_same_time_stamp=True
num_iter=4

In [8]:
man_mf=from_lobster.ManipulateMessageFile(
     LOB,messagefile,
     symbol=symbol,
     date=date,
     aggregate_time_stamp=aggregate_time_stamp,
     eventTypes_to_aggregate=eventTypes_to_aggregate,  
     only_4_events=only_4_events,
     separate_directions=separate_directions,
     separate_13_events=separate_13_events,
     separate_31_events=separate_31_events,
     separate_41_events=separate_41_events,
     separate_34_events=separate_34_events,
     separate_43_events=separate_43_events,
     equiparate_45_events_with_same_time_stamp=
     equiparate_45_events_with_same_time_stamp,
     eventTypes_to_drop_with_nonunique_time=eventTypes_to_drop_with_nonunique_time,
     eventTypes_to_drop_after_aggregation=eventTypes_to_drop_after_aggregation,
     drop_all_type3_events_with_nonunique_time=drop_all_type3_events_with_nonunique_time,  
     drop_5_events_with_same_time_stamp_as_4=drop_5_events_with_same_time_stamp_as_4,
     drop_5_events_after_aggregation=drop_5_events_after_aggregation,
     tolerance_when_dropping=tolerance_when_dropping,
     clear_same_time_stamp=clear_same_time_stamp,
     add_hawkes_marks=add_hawkes_marks,
     num_iter=num_iter
)

Initial length of the message file: 448996
prepare_from_lobster.check_time_nondecreasing: check passed successfully
ManipulateMessageFile.aggregate_time_stamp: contraction. only_4_events=False
  From 54805 lines with same time stamp, same event type and same direction, -->  to 24980 lines: 29825 lines contracted
  Initial length=448996, contracted length=419171
  Number of remaining events with same time stamp=3203
I am dropping events with non-unique time stamp and event type=3, while keeping the_last of the events in every batch
Numbers of events dropped: 94
I am dropping events with non-unique time stamp and event type=5, while keeping the_last of the events in every batch
Numbers of events dropped: 235
Total number of events dropped: 329
prepare_from_lobster.separate_41_events: number of occurrencies = 147 
   After separation, the average time step is 0.1295933537422549, the smallest time step is 1.4099350664764643e-07.
prepare_from_lobster.check_time_nondecreasing: check passed s

I am dropping events with non-unique time stamp and event type=7, while keeping none of the events in every batch
Numbers of events dropped: 0
Total number of events dropped: 4
prepare_from_lobster.check_time_nondecreasing: check passed successfully
event_encoding: self.n_event_types=4, self.n_levels=2


In [9]:
man_ob=from_lobster.ManipulateOrderBook(
    man_mf.LOB_sdhawkes,symbol=symbol,date=date,
    ticksize=man_mf.ticksize,n_levels=man_mf.n_levels,volume_imbalance_upto_level=2)

In [10]:
data=from_lobster.DataToStore(man_ob,man_mf)

prepare_from_lobster.check_time_nondecreasing: check passed successfully


In [11]:
sym = data.symbol
date = data.date
t_0 = data.initial_time
t_1 = data.final_time
print('Data are being stored in {}'.format(path_to_data))
with open(path_to_data+'/{}/{}_{}_{}-{}'.format(sym,sym,date,t_0,t_1), 'wb') as outfile:
    pickle.dump(data,outfile)
    

Data are being stored in /home/claudio/Desktop/lobster_for_sdhawkes/data/


In [12]:
model=sd_hawkes_model.sd_hawkes(number_of_event_types=data.number_of_event_types,  number_of_states = data.number_of_states,
 number_of_lob_levels=data.n_levels,volume_imbalance_upto_level = data.volume_enc.volume_imbalance_upto_level,
 list_of_n_states=data.state_enc.list_of_n_states, st1_deflationary=data.state_enc.st1_deflationary,
 st1_inflationary=data.state_enc.st1_inflationary, st1_stationary=data.state_enc.st1_stationary
)

In [13]:
model.get_input_data(data)

In [14]:
volumes=data.observed_volumes
states=data.observed_states
number_of_states=15
n_levels=2
verbose=True
tolerance=1.0e-10

In [15]:
import mle_estimation as mle_estim

In [16]:
dir_param=mle_estim.estimate_dirichlet_parameters(number_of_states,
                    n_levels,states,volumes,tolerance,verbose)

I am estimating the dirichlet parameters for the state 0
estimate_dirichlet_parameters. attempt with tol=1.000000013351432e-10
estimate_dirichlet_parameters. state=0. success with tol=1.000000013351432e-10
I am estimating the dirichlet parameters for the state 1
estimate_dirichlet_parameters. attempt with tol=1.000000013351432e-10
estimate_dirichlet_parameters. state=1. success with tol=1.000000013351432e-10
I am estimating the dirichlet parameters for the state 2
estimate_dirichlet_parameters. attempt with tol=1.000000013351432e-10
estimate_dirichlet_parameters. state=2. success with tol=1.000000013351432e-10
I am estimating the dirichlet parameters for the state 3
estimate_dirichlet_parameters. attempt with tol=1.000000013351432e-10
estimate_dirichlet_parameters. state=3. success with tol=1.000000013351432e-10
I am estimating the dirichlet parameters for the state 4
estimate_dirichlet_parameters. attempt with tol=1.000000013351432e-10
estimate_dirichlet_parameters. state=4. success w

In [17]:
dir_param

array([[0.7039276 , 0.36663016, 1.88691266, 0.45468069],
       [1.76764784, 1.56410014, 3.57362011, 2.19965356],
       [1.80324913, 3.47610326, 5.03950499, 4.32779329],
       [0.99427272, 7.04649004, 6.17537129, 7.86478626],
       [0.46064678, 6.06483743, 2.22097552, 5.65521195],
       [1.68951985, 0.35560893, 1.70843713, 0.68898937],
       [5.16692545, 1.72611975, 6.66723808, 4.65339844],
       [4.01889811, 4.15387306, 7.77893849, 7.60201168],
       [1.64306015, 4.93897685, 4.27448464, 5.9899214 ],
       [0.55798933, 2.81555622, 1.41841454, 3.3819862 ],
       [2.97722574, 0.34863461, 2.76921908, 1.17660321],
       [6.95127906, 1.04354641, 7.51660452, 5.62025043],
       [3.64939434, 1.98604595, 4.40513898, 5.11786108],
       [1.78843054, 2.11067483, 2.3892477 , 3.9269236 ],
       [1.02285726, 2.99258476, 1.18962651, 6.83080786]])